In [1]:
import requests
import re
from bs4 import BeautifulSoup
import json
import urllib

In [2]:
# s = "https://yandex.ru/maps/239/sochi/stops/stop__9973362/"
city_code = '239'
city_name = 'sochi'
stop = 'stop__9973362'

s0 = 'href="/maps/'+str(city_code)+'/'+city_name+'/routes/'

s = 'https://yandex.ru/maps/{}/{}/stops/{}/'.format(city_code, city_name, stop)

In [3]:
url = s

In [4]:
req = requests.get(url)
bs = BeautifulSoup(req.text, 'html.parser')

In [5]:
lst_links = bs.find_all('a')

In [6]:
lst_routes_links = []
route_names = []
titles = []
for link in lst_links:
    if s0 in str(link) and link.text != 'Маршруты':
        lst_routes_links.append('https://yandex.ru'+link.get('href'))
        route_names.append(link.text)
        titles.append(link.get('title'))
# 

In [7]:
url1 = lst_routes_links[0]

In [8]:
req = requests.get(url1)
bs = BeautifulSoup(req.text, 'html.parser')

In [9]:
scrpt = bs.find_all('script')[-2].text
js = json.loads(scrpt)

In [10]:
js['masstransitLine']['features'][0]['features'][-1]

{'coordinates': [39.773594992, 43.557614811],
 'id': 'stop__9973554',
 'name': 'Микрорайон Искра',
 'region': {'bounds': [[39.1261495, 43.261256799670214],
   [40.3199745, 43.908055799670215]],
  'hierarchy': [225, 10995, 239],
  'id': 239,
  'seoname': 'sochi'}}

In [11]:
bs.find_all('script')[-2].get("type") == "application/json"

True

In [14]:
all_scrpt = bs.find_all('script')
for scr in all_scrpt:
    if scr.get("type") == "application/json":
        print(all_scrpt.index(scr))
# 

12


In [16]:
str1 = 'остановка общественного транспорта'
str_qt = urllib.parse.quote(str1)

In [20]:
srch = "https://yandex.ru/maps/239/sochi/search/{}".format(str_qt)

In [21]:
req = requests.get(srch)
bs = BeautifulSoup(req.text, 'html.parser')

In [33]:
all_scrpt = bs.find_all("script")

In [34]:
all_scrpt = bs.find_all("script")
for scr in all_scrpt:
    if scr.get('type') == "application/json":
        print(all_scrpt.index(scr))

14


In [28]:
scr1 = all_scrpt[13].text
js2 = json.loads(scr1)

In [32]:
js2.keys()

dict_keys(['counters', 'hosts', 'locale', 'lang', 'tld', 'query', 'baseUrl', 'originalUrl', 'uatraits', 'cspNonce', 'origin', 'csrfToken', 'xscriptCsrfToken', 'isUserFromPortal', 'isUserFromGoogle', 'availableLangs', 'currencyCode', 'banner', 'staticHost', 'pageMode', 'bookmarks', 'clientErrors', 'searchResultsCount', 'enableTaxi', 'enableAvia', 'enableZapravkiPromo', 'iframeCounter', 'tnsCounter', 'cloudApi', 'blaBlaCarBanner', 'search', 'routes', 'feedback', 'font', 'advert', 'serviceName', 'bilet', 'share', 'mobile', 'servicePanel', 'apiManual', 'mapMargin', 'business', 'troubleshooting', 'passport', 'morda', 'ruler', 'ugcCabinet', 'advertLandingUrl', 'advertLandingMetroUrl', 'becomeAuthorUrl', 'becomeOwnerUrl', 'agreementUrl', 'metroAgreementUrl', 'rightHoldersUrl', 'ymapsUrlTemplate', 'mrcLandingUrl', 'reviews', 'enableRoutesB2B', 'routesB2BUrl', 'enableMobileDistribution', 'bannerLayout', 'pharmacyLinks', 'mapLocation', 'mapType', 'userRegion', 'experiments', 'layerLinks', 'layer

In [34]:
js2['searchPreloadedResults'].keys()

dict_keys(['requestId', 'requestSerpId', 'requestContext', 'requestQuery', 'requestCorrectedQuery', 'requestBounds', 'displayType', 'totalResultCount', 'requestResults', 'requestSkip', 'items', 'bounds', 'categories', 'filters', 'hasLavkaBanner'])

In [48]:
js2['searchPreloadedResults']['items'][1].keys()

dict_keys(['type', 'requestId', 'analyticsId', 'title', 'description', 'address', 'coordinates', 'displayCoordinates', 'bounds', 'logId', 'uri', 'id', 'metro', 'stops', 'panorama', 'shortTitle', 'fullAddress', 'country', 'status', 'businessLinks', 'ratingData', 'sources', 'categories', 'businessProperties', 'modularSnippet', 'seoname', 'geoId', 'compositeAddress', 'references', 'subtitleItems', 'region', 'breadcrumbs'])

In [53]:
js2['searchPreloadedResults']['items'][1]['description']

'Россия, Краснодарский край, Сочи, улица Конституции СССР'

# Способ поиска всех маршрутов в городе по названию

In [ ]:
import pandas as pd
import shapely
from shapely.geometry import Point, LineString, MultiLineString

In [80]:
str_ct = 'Самара'
str_qt = urllib.parse.quote(str_ct)
srch_ct = "https://yandex.ru/maps/?text={}".format(str_qt)

In [81]:
req = requests.get(srch_ct)
bs = BeautifulSoup(req.text, 'html.parser')

In [83]:
alla = bs.find_all('a')
for aa in alla:
    cls = aa.get("class")
    try:
        if 'catalog-entry-point' in cls:
            hrf = aa.get("href")
            break
    except:
        pass
# 
try:
    city_id,city_name = hrf.split("maps/")[1].split("/catalog")[0].split("/")
except:
    city_id,city_name = None,None
    print("Error. Not found")
    pass
#


In [85]:
lnk_tp = "https://yandex.ru/maps/"+ str(city_id) + "/" + str(city_name) + "/transport"

In [96]:
def find_js(lnk):
    req = requests.get(lnk)
    bs = BeautifulSoup(req.text, 'html.parser')
    all_scrpt = bs.find_all("script")
    for scr in all_scrpt:
        if scr.get('type') == "application/json":
            sc_ind = all_scrpt.index(scr)
            break
    # 
    scrp_txt = all_scrpt[sc_ind].text
    js = json.loads(scrp_txt)
    
    return js

In [105]:
js_tp = find_js(lnk_tp)

In [ ]:
link

In [379]:
req = requests.get(lnk_tp)
bs = BeautifulSoup(req.text, 'html.parser')
lst_srp = bs.find_all('script')

In [384]:
lst_srp[11]

<script type="application/ld+json">{"@context":"http://schema.org","@type":"BreadcrumbList","itemListElement":[{"@type":"ListItem","position":1,"item":{"@id":"https://yandex.ru/maps/","name":"Карты"}},{"@type":"ListItem","position":2,"item":{"@id":"https://yandex.ru/maps/51/samara/","name":"Самара"}}]}</script>

In [387]:
lst_lineid = []
lst_tpts = []
lst_names = []
lst_rts = []
i=0
for i in range(len(js_tp['vehicles'])):
    one_lst = []
    try:
        meta_veh = js_tp['vehicles'][i]['properties']['VehicleMetaData']['Transport']
        lineid = meta_veh['lineId']
        if lineid not in lst_lineid:
            lst_lineid.append(lineid)
            one_lst.append(lineid)
            answ_line = 1
        else:
            answ_line = 0
        try:
            transp_type = meta_veh['type']
            name_rt = meta_veh['name']
        except:
            transp_type,name_rt = None,None
        if answ_line == 1:
            lst_tpts.append(transp_type)
            lst_names.append(name_rt)
            one_lst.append(name_rt)
            one_lst.append(transp_type)
        #
        if one_lst != []:
            lst_rts.append(one_lst)
    except:
        pass
# 

In [390]:
lst_rts[0][0]

'2193179075'

In [277]:
dct_lnks = {}
i=0
# for i in range(len(lst_rts)):
for i in range(5):
#     lnkid = lst_rts[i][0]
    lnkid = lst_lineid[i]
    lnk_rt = 'https://yandex.ru/maps/{}/{}/routes/{}'.format(city_id, city_name, lnkid)
    js_rt = find_js(lnk_rt)
    dct_lnks[lnkid] = js_rt['masstransitLine']['features']
# 

In [377]:
js_rt['meta']

{'title': 'Трамвай 20: маршрут, остановки и расписание в Самаре',
 'breadcrumbs': [{'name': 'Карты',
   'type': 'root',
   'url': 'https://yandex.ru/maps/'},
  {'name': 'Самара',
   'type': 'region',
   'url': 'https://yandex.ru/maps/51/samara/',
   'region': {'center': [50.101783, 53.195538], 'zoom': 12}},
  {'name': 'Общественный транспорт',
   'type': 'masstransit-home',
   'url': 'https://yandex.ru/maps/51/samara/transport/'},
  {'name': 'Трамваи',
   'type': 'masstransit-list',
   'url': 'https://yandex.ru/maps/51/samara/transport/trams/',
   'vehicleType': 'tramway'},
  {'name': 'Трамвай 20',
   'type': 'search',
   'url': 'https://yandex.ru/maps/51/samara/routes/tramway_20/796d617073626d313a2f2f7472616e7369742f6c696e653f69643d32313933313739343031266c6c3d35302e31353030313025324335332e323132353431266e616d653d323026723d3532363426747970653d7472616d776179/',
   'currentPage': True}]}

In [373]:
# cnt=1
# for elem in dct_lnks:
#     cnt+=1
#     print(elem)
#     if cnt==2:
#         break

2193179075


In [ ]:
# def get_rt_data(one_rt,rt_id):
## one_rt = dct_lnks[elem]
## rt_id = elem

data_frw = get_dir_data(one_rt[0])
stp_frw = one_rt[0]['properties']['ThreadMetaData']['EssentialStops'][0]['name']
stp_bckw = one_rt[0]['properties']['ThreadMetaData']['EssentialStops'][1]['name']
str_rt_nm = str(stp_frw) + "_" str(stp_bckw)

if len(one_rt) == 2:
    data_bcw = get_dir_data(one_rt[1])
    is_circle = False
    
else:
    is_circle = True
    
# 

In [374]:
one_rt.keys()

AttributeError: 'list' object has no attribute 'keys'

In [364]:
one_rt = dct_lnks['2193179075']

In [372]:
one_rt[0]['properties']['ThreadMetaData']['EssentialStops'][0]['name']

'Овощная база'

In [369]:
one_rt[0]

{'features': [{'id': 'stop__10002140',
   'coordinates': [50.2948622, 53.253455],
   'name': 'Овощная база',
   'region': {'id': 51,
    'hierarchy': [225, 11131, 51],
    'seoname': 'samara',
    'bounds': [[49.772227, 52.96536237206464],
     [50.431338999999994, 53.42448437206464]]}},
  {'points': [[50.294937, 53.253422],
    [50.294779, 53.253065],
    [50.294661999999995, 53.252776999999995],
    [50.294615, 53.252646],
    [50.294494, 53.252264],
    [50.294461999999996, 53.252168999999995],
    [50.29445, 53.252061999999995],
    [50.294453, 53.252002],
    [50.294475999999996, 53.251912999999995],
    [50.294497, 53.251855],
    [50.294602, 53.251692],
    [50.294837, 53.251368],
    [50.294886, 53.251279999999994],
    [50.29492, 53.251219999999996],
    [50.294940999999994, 53.251169],
    [50.294954999999995, 53.251118],
    [50.294965, 53.251068],
    [50.294954999999995, 53.251014999999995],
    [50.294928999999996, 53.250969],
    [50.294897999999996, 53.250929],
    [50.

In [278]:
dct_lnks.keys()

dict_keys(['2193179075', '2193179452', '2193179408', '2193179080', '2193179401'])

In [280]:
len(dct_lnks['2193179075'])

2

In [ ]:
one_rt = dct_lnks['2193179075']

In [363]:
js_rt['masstransitLine']['features'][0].keys()

dict_keys(['features', 'properties', 'MapsUIMetaData'])

In [ ]:
dct_lnks['2193179075'][]

In [ ]:
# one_rt[0]['properties']['ThreadMetaData']['EssentialStops']

In [ ]:
# for elem in dct_lnks:
#     frw = elem[0]
#     if len(elem) == 2:
#         bckw = elem[1]
#     else:
#         bckw = None
# # 

In [281]:
one_rt = dct_lnks['2193179075']

In [305]:
one_dir = one_rt[0]['features']

In [ ]:
def get_stop(dct):
    lst=[]
    stop_id = dct['id']
    name = dct['name']
    stop_lon = dct['coordinates'][0]
    stop_lat = dct['coordinates'][1]
    lst = [stop_id,name,stop_lat,stop_lon]
    return lst

In [357]:
def get_dir_data(one_dir,direction,rt_id):
    all_stops=[]
    rt_line=[]
    line_data=[]
    i=0
    lst_stop=[]
    cnt_stp = 0
    line_data = line_data + [rt_id,direction]
    for i in range(len(one_dir)):
        if 'id' in one_dir[i]:
            cnt_stp +=1
            lst_stop = []
            lst_stop = lst_stop + [rt_id,direction,cnt_stp]
            lst_stop = lst_stop + get_stop(one_dir[i])
            all_stops.append(lst_stop)

        if 'points' in one_dir[i]:
            one_part = one_dir[i]['points']
            for j in range(len(one_part)):
                rt_line.append(one_part[j])
        #
    # 
    line_data.append(LineString(rt_line))
    
    return all_stops,line_data

In [358]:
direction = 0
rt_id = '2193179075'
all_stops,line_data = get_dir_data(one_dir,direction,rt_id)

In [336]:
import pandas as pd
import shapely
from shapely.geometry import Point, LineString, MultiLineString

In [ ]:
# "https://yandex.ru/maps/51/samara/transport/buses/?l=masstransit&ll=50.212459%2C53.227108&z=11"

In [ ]:
# https://yandex.ru/maps/51/samara/transport/?ll=50.183735%2C53.233963&z=10